In [68]:
import pandas as pd
import numpy as np
from scipy.spatial import ConvexHull
import matplotlib.pyplot as plt
import math
import scipy.linalg

In [69]:
from sympy import *
init_printing()

x = Symbol('x')
y = Symbol('y')
a = Symbol('a')
y = Symbol('b')


#sbl = Matrix(symbols('x,y,a,b'))
sbl = Matrix(symbols('x,y'))


In [70]:
def Funcion_Rosenbrok(x,a=3,b=2):
    resultado=(a-x[0])**2+b*(x[1]-x[0]**2)**2
    
    return resultado

In [ ]:
def Hess(f, x0, h=1e-4, method = "basic"):
    """
    Función que calcula la Hessiana  de una función en un punto. 
    f: función sobre la cual queremos calcular la hessiana.
    x0: Punto sobre el cual queremos hacer el cálculo
    h: nivel de precisión para hacer el cálculo
    method: Método por el cual se quiere hacer puede ser: 'basic', 'grad', 'centered', 'gradCentered'
    """
    n = len(x0)
    Hess = np.matrix(np.zeros((n,n)))
    for i in range(n):
        for j in range(n):
            z_i = np.zeros(n)
            z_i[i] = h
            z_j = np.zeros(n)
            z_j[j] = h
            if method == "basic":
                Hess[i,j] = ( f(x0 + z_j +z_i) - f(x0 + z_i ) - f(x0+z_j) +f(x0)) / (h**2)
            elif method == "grad":
                Hess[i,j] = (Grad(f,x0+z_j,h,i) - Grad(f,x0,h,i) + \
                             Grad(f,x0+z_i,h,j) - Grad(f,x0,h,j))/(2*h)
            elif method == "centered":
                if i==j:
                    Hess[i,j] = (-f(x0+2*z_i) + 16*f(x0+z_i) - 30*f(x0)+\
                                 16*f(x0-z_i) - f(x0-2*z_i))  / (12*h**2)
                else :
                    Hess[i,j] = (f(x0+z_i+z_j) - f(x0 + z_i - z_j) - \
                                 f(x0 - z_i + z_j) + f(x0-z_i-z_j))/(4*h**2)
            elif method == "gradCentered":
                    Hess[i,j] = (Grad(f,x0+z_j,h)[i] - Grad(f, x0-z_j,h)[i] + \
                                 Grad(f,x0+z_i,h)[j] - Grad(f,x0-z_i,h)[j])/(4*h)
    return Hess

In [ ]:
def Grad(f, x0, h=1e-6, i=-1):
    """
    Función que calcula el Grad de una función en un punto
    """
    n = len(x0)
    if i in range(n):
        z = np.zeros(n)
        z[i] = h/2
        Grad = (f(x0 + z) - f(x0 - z))/h
    else:
        Grad=np.zeros(n)
        for j in range(n):
            z = np.zeros(n)
            z[j] = h/2
            Grad[j]= (f(x0 + z) - f(x0 - z))/h
    return Grad

In [ ]:
def condiciones_wolfe(f, x0, alpha, pk, c1=1e-4, c2 = 1e-2, tol=1e-5):
    """
    Función que evalúa las condiciones de wolfe para una alpha. 
    f:  función que optimizamos
    x0: punto anterior un numpy.array
    alpha: valor que cumplirá condiciones de wolfe. 
    pk: dirección de decenso un numpy.array
    """
    grad = lambda alpha: Grad(f,x0+alpha*pk, tol)
    phi = lambda alpha: f(x0 + alpha*pk) # Ojo que phi(0) = f(x0)
    linea = lambda alpha: phi(0) + c1 * alpha *np.dot( g_x0, pk)
    g_x0 = grad(0) # grad(0) = Grad(f,x0)
    cond_1 = linea(alpha) - phi(alpha) >= 0
    cond_2 = np.dot(grad(alpha), pk) - c2 * np.dot(g_x0, pk) >=0
    return  cond_1 and cond_2 

In [ ]:
def genera_alpha(f,x0,pk,c1=1e-4,c2=1e-2,tol=1e-5):
    """
    Backtracking LS. i.e Algoritmo que encuentra una alpha que cumple con diciones de wolfe
    """
    alpha,rho,c=1,4/5,c1
    rho=4/5
    while f(x0+alpha*pk)> f(x0)+c*alpha*np.dot(Grad(f,x0),pk):
        alpha*=rho
    
    return alpha

In [71]:
#Gradiente simbolico
def GradSim(f, variables):
    return Matrix([f]).jacobian(Matrix(symbols(variables))).T

In [72]:
a=3
b=4

In [73]:
#Hessiana Simbolica
def Hessian(f, variables):
    g = GradSim(f,variables)
    return GradSim(g.T, variables)

In [74]:
#Funcion Symbolica
f = (a-x)**2+b*(y-x**2)**2

In [75]:
Grad(f, 'x y ')

⎡       ⎛     2⎞          ⎤
⎢- 16⋅x⋅⎝b - x ⎠ + 2⋅x - 6⎥
⎢                         ⎥
⎣            0            ⎦

In [76]:
Hessian(f, 'x y')

⎡            2       ⎤
⎢-16⋅b + 48⋅x  + 2  0⎥
⎢                    ⎥
⎣        0          0⎦

In [114]:
def Newton(f,x0,c1=1e-4,c2=1e-2,tol=1e-5):
  
    k=0
    n = len(x0)
    xk=x0
    pk=-(Grad(f, xk, h=1e-6, i=-1))
    alpha_k=1
    while    not(condiciones_wolfe(f,xk,alpha_k,pk,c1=1e-6, c2 = 1e-6, tol=1e-6) ):#and condicionPrimerOrden(f,xk,0.001)):
        
        xk=xk+alpha_k*pk
        alpha_k=1
        pk=-(Grad(f, xk, h=1e-6, i=-1))
     
        k=k+1
    
    return xk,k

In [115]:
p=np.array([20,20**2])
Newton(Funcion_Rosenbrok,p)

(array([  9.51687040e+13,   1.98399958e+09]), 3)

In [118]:
#Algoritmo de Busqueda lineal
def busqueda_lineal(f,x0,c1=1e-4,c2=1e-2,tol=1e-5):
    """
    Aplica el algoritmo de busqueda lineal y te aproxima el optimo, condicionado a las expreciones de Wolfe 
    
    """
    k=0
    n = len(x0)
    xk=x0
    pk=-(Grad(f, xk, h=1e-6, i=-1))
    alpha_k=0.001
    while   not (condiciones_wolfe(f,xk,alpha_k,pk,c1=1e-6, c2 = 1e-6, tol=1e-6) ):#and condicionPrimerOrden(f,xk,0.001)):
        
        xk=xk+alpha_k*pk
        
        pk=-(Grad(f, xk, h=1e-6, i=-1))
     
        k=k+1
    
    return xk,k

In [119]:
p=np.array([20,20**2])
busqueda_lineal(Funcion_Rosenbrok,p)

(array([  1.03950581e+13,   6.31531204e+02]), 9)

In [105]:
t=Grad(Funcion_Rosenbrok,p)

In [106]:
genera_alpha(Funcion_Rosenbrok,p,t,c1=1e-6,c2=1e-5,tol=1)